In [1]:
# The goal is to show that when using a random ranking feature the final `relevance` score is not equal to the last ranking phase.

In [2]:
!docker run --detach --rm --name vespa-randrank --hostname vespa-randrank --publish 0.0.0.0:8080:8080 --publish 0.0.0.0:19050:19050 --publish 0.0.0.0:19071:19071 vespaengine/vespa:8.548.23

07901bf2c87a7ee2bca341053474ee83d74814e997c69aab222bf0fa00d47e96


In [8]:
# Setup a simple Vespa application package
from vespa.package import (ApplicationPackage, Field, Schema, Document, RankProfile, Function,
                           QueryProfile, QueryProfileType, FirstPhaseRanking)
from vespa.io import VespaResponse

vap = ApplicationPackage(
    name="randrank",
    query_profile_type=QueryProfileType(
        fields=[]
    ),
    query_profile=QueryProfile(
        fields=[]
    ),
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(
                        name="id",
                        type="int",
                        indexing=["attribute", "summary"],
                        attribute=["fast-search"],
                    ),
                ]
            ),
            rank_profiles=[
                RankProfile(
                    name="randrank",
                    functions=[
                        Function(
                            name='fuzziness_boost',
                            expression='random(attribute(filter))',
                        ),
                        Function(
                            name='final_ranking_score',
                            expression='fuzziness_boost + 1'
                        )
                    ],
                    first_phase=FirstPhaseRanking(
                        expression='final_ranking_score',
                    ),
                    match_features=['final_ranking_score', 'fuzziness_boost', 'firstPhase'],
                    summary_features=['final_ranking_score', 'fuzziness_boost', 'firstPhase'],
                ),
            ]
        )
    ]
)

In [9]:
vap_file_name = "randrank.zip"
vap.to_zipfile(vap_file_name)
! vespa deploy {vap_file_name} -t http://localhost:19071

Uploading application package... done;1m⣟
Success: Deployed 'randrank.zip' with session ID 3


In [10]:
from vespa.application import Vespa

prod_vespa_host = "http://localhost"
app = Vespa(url=prod_vespa_host, port=8080)

In [11]:
# Create and feed 5 docs
docs = [
    {
        'id': f'{i}',
        'fields': {
            'id': i,
        }
    } for i in range(5)]


def callback(response: VespaResponse, document_id: str):
    if not response.is_successful():
        print(f"Error when feeding document {document_id}: {response.get_json()}")


app.feed_iterable(docs, schema="doc", namespace="doc", callback=callback)

In [12]:
app.query(body={
    'yql': 'select matchfeatures, summaryfeatures '
           'from sources * '
           'where true',
    'ranking.profile': 'randrank',
}).json

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 5},
  'coverage': {'coverage': 100,
   'documents': 5,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'index:randrank_content/0/c81e728d3cb3a054bb332a60',
    'relevance': 1.6379597610794008,
    'source': 'randrank_content',
    'fields': {'matchfeatures': {'firstPhase': 1.6886327285319567,
      'final_ranking_score': 1.6886327285319567,
      'fuzziness_boost': 0.6886327285319567},
     'summaryfeatures': {'firstPhase': 1.0899378973990679,
      'final_ranking_score': 1.0899378973990679,
      'fuzziness_boost': 0.08993789739906788,
      'vespa.summaryFeatures.cached': 0.0}}},
   {'id': 'index:randrank_content/0/a87ff679afc128dfacf8e944',
    'relevance': 1.4935250622220337,
    'source': 'randrank_content',
    'fields': {'matchfeatures': {'firstPhase': 1.1759059918113053,
      'final_ranking_score': 1.1759059918113053,
      'fuzziness_boost': 0.17590599181130

In [ ]:
!docker stop vespa-randrank